In [15]:
import pandas as pd
from sklearn.metrics import mean_squared_error
import numpy as np

df = pd.read_csv('random_forest_predictions_with_actuals.csv')
print(df.head())

                    run_date            target_datetime  predicted_Price  \
0  2025-03-15 00:00:00+00:00  2025-03-16 01:00:00+00:00         0.093036   
1  2025-03-15 00:00:00+00:00  2025-03-16 02:00:00+00:00         0.091816   
2  2025-03-15 00:00:00+00:00  2025-03-16 03:00:00+00:00         0.092445   
3  2025-03-15 00:00:00+00:00  2025-03-16 04:00:00+00:00         0.093663   
4  2025-03-15 00:00:00+00:00  2025-03-16 05:00:00+00:00         0.093793   

   actual_Price  oxygent_price  naive_price  
0       0.08579          0.083      0.08680  
1       0.07962          0.079      0.08570  
2       0.07818          0.074      0.09586  
3       0.07860          0.073      0.09772  
4       0.08051          0.076      0.09985  


# GENERAL PERFORMANCE METRICS


In [16]:
# Calculate missing values count and percentage
missing_values = pd.DataFrame({
    'Missing Count': df.isnull().sum(),
    'Missing Percentage': (df.isnull().sum() / len(df) * 100).round(2)
})
print(missing_values)

                 Missing Count  Missing Percentage
run_date                     0                0.00
target_datetime              0                0.00
predicted_Price              0                0.00
actual_Price                 0                0.00
oxygent_price                6                0.14
naive_price                  0                0.00


In [17]:
df['oxygent_price'] = df['oxygent_price'].fillna(df['predicted_Price'])

In [18]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# Calculate metrics for each model
results = {
    'Model': ['Random Forest', 'Oxygent', 'Naive'],
    'RMSE': [
        np.sqrt(mean_squared_error(df['actual_Price'], df['predicted_Price'])),
        np.sqrt(mean_squared_error(df['actual_Price'], df['oxygent_price'])),
        np.sqrt(mean_squared_error(df['actual_Price'], df['naive_price']))
    ],
    'MAE': [
        mean_absolute_error(df['actual_Price'], df['predicted_Price']),
        mean_absolute_error(df['actual_Price'], df['oxygent_price']),
        mean_absolute_error(df['actual_Price'], df['naive_price'])
    ]
}

# Create results DataFrame and round to 6 decimal places
results_df = pd.DataFrame(results).round(6)
print(results_df)

           Model      RMSE       MAE
0  Random Forest  0.035374  0.025476
1        Oxygent  0.077176  0.058853
2          Naive  0.042212  0.030525


In [19]:
import numpy as np
# Count number of samples per forecast horizon
df['forecast_horizon'] = np.ceil((pd.to_datetime(df['target_datetime']) - pd.to_datetime(df['run_date'])).dt.total_seconds()/(24*3600))
print(df['forecast_horizon'].value_counts().sort_index())

forecast_horizon
2.0    720
3.0    720
4.0    720
5.0    720
6.0    720
7.0    720
Name: count, dtype: int64


In [22]:

# Initialize empty dictionary with lists
horizon_results = {
    'Horizon': [],
    'Random Forest': [],
    'Oxygent': [],
    'Naive': []
}

# Fill lists with results
for h in horizons:
    horizon_data = df[df['forecast_horizon'] == h]
    horizon_results['Horizon'].append(h)
    horizon_results['Random Forest'].append(np.sqrt(mean_squared_error(horizon_data['actual_Price'], horizon_data['predicted_Price'])))
    horizon_results['Oxygent'].append(np.sqrt(mean_squared_error(horizon_data['actual_Price'], horizon_data['oxygent_price'])))
    horizon_results['Naive'].append(np.sqrt(mean_squared_error(horizon_data['actual_Price'], horizon_data['naive_price'])))

# Create and format results DataFrame
horizon_results_df = pd.DataFrame(horizon_results).set_index('Horizon').round(6)
print(horizon_results_df)


         Random Forest   Oxygent     Naive
Horizon                                   
2             0.035640  0.074895  0.042605
3             0.035503  0.082894  0.042836
4             0.036082  0.077497  0.042669
5             0.034965  0.078746  0.042153
6             0.033881  0.076152  0.041907
7             0.036122  0.072460  0.041073


In [23]:
# Calculate bias and metrics for each model
bias_results = {
    'Model': ['Random Forest', 'Oxygent', 'Naive'],
    'Bias': [
        (df['predicted_Price'] - df['actual_Price']).mean(),
        (df['oxygent_price'] - df['actual_Price']).mean(),
        (df['naive_price'] - df['actual_Price']).mean()
    ],
    'MAE': [
        mean_absolute_error(df['actual_Price'], df['predicted_Price']),
        mean_absolute_error(df['actual_Price'], df['oxygent_price']),
        mean_absolute_error(df['actual_Price'], df['naive_price'])
    ],
    'RMSE': [
        np.sqrt(mean_squared_error(df['actual_Price'], df['predicted_Price'])),
        np.sqrt(mean_squared_error(df['actual_Price'], df['oxygent_price'])),
        np.sqrt(mean_squared_error(df['actual_Price'], df['naive_price']))
    ]
}

# Create DataFrame and round results to 6 decimal places
bias_results_df = pd.DataFrame(bias_results).round(6)
print(bias_results_df)

           Model      Bias       MAE      RMSE
0  Random Forest  0.003132  0.025476  0.035374
1        Oxygent  0.006267  0.058853  0.077176
2          Naive  0.006511  0.030525  0.042212


De **bias**-waarden geven aan of een model structureel te hoog of te laag voorspelt ten opzichte van de werkelijke waarde:

- **Bias = gemiddeld(Voorspelling - Werkelijk)**  
  - Een **positieve bias** betekent dat het model gemiddeld te hoge waarden voorspelt (overschatting).
  - Een **negatieve bias** betekent dat het model gemiddeld te lage waarden voorspelt (onderschatting).
  - Een **bias dichtbij 0** betekent dat het model gemiddeld genomen geen structurele overschatting of onderschatting heeft.

**Voorbeeld:**  
- Bias van +2: het model voorspelt gemiddeld 2 eenheden te hoog.
- Bias van -1.5: het model voorspelt gemiddeld 1.5 eenheden te laag.

**Let op:**  
Een lage bias betekent niet automatisch dat het model goed is; het kan nog steeds grote fouten maken (hoge MAE of RMSE), zolang de overschattingen en onderschattingen elkaar maar ongeveer opheffen. Bias zegt dus alleen iets over de richting van de systematische fout, niet over de spreiding van de fouten.

# PERFORMANCE RELATIVE TO MOST EXPENSIVE HOURS, WITHIN EACH RUN (SO 6 DAYS HORIZON)

In [24]:
# Initialize results dictionary
comparison_results = {
    'model': [],
    'avg_overlap_cheap': [],
    'avg_overlap_expensive': [],
    'avg_price_diff_cheap': [],
    'avg_price_diff_expensive': []
}

# Get unique run dates
run_dates = df['run_date'].unique()

# Analyze each run_date
for run_date in run_dates:
    # Get data for current run date
    day_data = df[df['run_date'] == run_date]
    
    # Find actual cheapest and most expensive hours
    actual_cheap = day_data.nsmallest(4, 'actual_Price')
    actual_expensive = day_data.nlargest(4, 'actual_Price')
    
    # Calculate actual average prices for reference
    actual_cheap_avg = actual_cheap['actual_Price'].mean()
    actual_expensive_avg = actual_expensive['actual_Price'].mean()
    
    # Analyze each model
    for model in ['predicted_Price', 'oxygent_price', 'naive_price']:
        model_name = 'Random Forest' if model == 'predicted_Price' else ('Oxygent' if model == 'oxygent_price' else 'Naive')
        
        # Find model's predicted cheapest and most expensive hours
        model_cheap = day_data.nsmallest(4, model)
        model_expensive = day_data.nlargest(4, model)
        
        # Calculate overlaps
        cheap_overlap = len(set(model_cheap.index) & set(actual_cheap.index))
        expensive_overlap = len(set(model_expensive.index) & set(actual_expensive.index))
        
        # Calculate average prices for model's selections
        model_cheap_actual_avg = day_data.loc[model_cheap.index, 'actual_Price'].mean()
        model_expensive_actual_avg = day_data.loc[model_expensive.index, 'actual_Price'].mean()
        
        # Store results
        if model_name not in comparison_results['model']:
            comparison_results['model'].append(model_name)
            comparison_results['avg_overlap_cheap'].append([])
            comparison_results['avg_overlap_expensive'].append([])
            comparison_results['avg_price_diff_cheap'].append([])
            comparison_results['avg_price_diff_expensive'].append([])
        
        idx = comparison_results['model'].index(model_name)
        comparison_results['avg_overlap_cheap'][idx].append(cheap_overlap)
        comparison_results['avg_overlap_expensive'][idx].append(expensive_overlap)
        comparison_results['avg_price_diff_cheap'][idx].append(model_cheap_actual_avg - actual_cheap_avg)
        comparison_results['avg_price_diff_expensive'][idx].append(actual_expensive_avg - model_expensive_actual_avg)

# Calculate final averages
final_results = pd.DataFrame({
    'Model': comparison_results['model'],
    'Avg Overlap Cheap Hours': [np.mean(x) for x in comparison_results['avg_overlap_cheap']],
    'Avg Overlap Expensive Hours': [np.mean(x) for x in comparison_results['avg_overlap_expensive']],
    'Avg Price Diff Cheap (€)': [np.mean(x) for x in comparison_results['avg_price_diff_cheap']],
    'Avg Price Diff Expensive (€)': [np.mean(x) for x in comparison_results['avg_price_diff_expensive']]
}).round(4)

print(final_results)

           Model  Avg Overlap Cheap Hours  Avg Overlap Expensive Hours  \
0  Random Forest                   1.1333                       1.7667   
1        Oxygent                   1.4667                       0.4333   
2          Naive                   1.0333                       1.6000   

   Avg Price Diff Cheap (€)  Avg Price Diff Expensive (€)  
0                    0.0353                        0.0169  
1                    0.0212                        0.0542  
2                    0.0375                        0.0210  
